In [6]:
from google.cloud import bigquery
import pandas as pd

In [7]:

time = '2025-06-25'
SQL_QUERY =  f"""
SELECT
  sso_id,additional_info,campaign_details,campaign_name,platform,event_time
FROM
  `htz-common.raw_data.requests` 
WHERE
  TIMESTAMP_TRUNC(event_time, DAY) >= TIMESTAMP('{time}')
  AND page_type = 'Chatbot'
  AND campaign_name in ('response answer','open question')
  -- AND sso_id IS NOT NULL
ORDER BY
  event_time DESC;
"""
client = bigquery.Client()
query_job = client.query(SQL_QUERY)

results = query_job.result() 
df = results.to_dataframe()
print(df.shape[0])
df.head(2)

25


/Users/roeishlezinger/Desktop/movie-recommendation-chatbot/.venv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,sso_id,additional_info,campaign_details,campaign_name,platform,event_time
0,8010962861,"[{'key': 'total_time', 'value': '2.22433567047...","היוש! אני רני, פה כדי לעזור לכם למצוא את הסרט ...",response answer,Desktop,2025-06-25 05:55:57.888000+00:00
1,8010962861,"[{'key': 'session_id', 'value': '120'}]",היווווש,open question,Desktop,2025-06-25 05:55:52.116000+00:00


In [8]:
# preprocess the data

df['parsed_data'] = df['additional_info'].apply(lambda x: {item['key']: item['value'] for item in x})
expanded_df = pd.json_normalize(df['parsed_data'].tolist())
expanded_df = expanded_df.drop(columns=['sso_id'])
df_cleaned = df.drop(columns=['additional_info', 'parsed_data'])
df = pd.concat([df_cleaned, expanded_df], axis=1)
df['id'] = df['sso_id'].astype(str) + '_' + df['session_id'].astype(str)

df.head(2)

,sso_id,campaign_details,campaign_name,platform,event_time,total_time,rag_speed,session_id,timestamp,input_tokens,...,article_ids_1,error,remaining_user_messages,troll_triggered,conversation_key,genres,regenerate,media_type,streaming_platforms,id
0,8010962861,"היוש! אני רני, פה כדי לעזור לכם למצוא את הסרט ...",response answer,Desktop,2025-06-25 05:55:57.888000+00:00,2.2243356704711914,2.071857452392578E-4,120,1.7508309576415713E9,74,...,0000017f-e071-d804-ad7f-f1fbb77c0000,1,198,true,8010962861_120,,false,,,8010962861_120
1,8010962861,היווווש,open question,Desktop,2025-06-25 05:55:52.116000+00:00,NaN,NaN,120,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8010962861_120


In [9]:
df[df['campaign_details'].fillna('').str.contains('closing_question')]

,sso_id,campaign_details,campaign_name,platform,event_time,total_time,rag_speed,session_id,timestamp,input_tokens,...,article_ids_1,error,remaining_user_messages,troll_triggered,conversation_key,genres,regenerate,media_type,streaming_platforms,id


In [10]:
df['id'].value_counts().sort_values(ascending=False).head(10)

id
8010962861_117    12
8010962861_118    10
8010962861_120     2
8010962861_119     1
Name: count, dtype: int64

In [16]:
def parse_conversation(ids:str) -> str:
    """
    Parse the conversation from the DataFrame based on the provided id.
    """
    conversation = df[df['id'] == ids]
    conversation = conversation.sort_values(by='event_time', ascending=True)
    if conversation.empty:
        return "No conversation found for this ID."
    
    conversation_text = []
    for _, row in conversation.iterrows():
        if row['campaign_name'] == 'open question':
            text = f"{row['event_time']} - USER: {row['campaign_details']}"
        elif row['campaign_name'] == 'response answer':
            text = f"{row['event_time']} - LLM: {row['campaign_details']}"
        conversation_text.append(text)
    
    return "\n".join(conversation_text)

txt = parse_conversation('8010962861_117')
print(txt)

2025-06-25 04:47:50.808000+00:00 - USER: בוקר טוב
2025-06-25 04:48:17.441000+00:00 - USER: סרט באמזון. שיהיה טוב, אני פתוח לדרמות, קומדיות ומצוייר. לא מתח
2025-06-25 04:48:21.942000+00:00 - LLM: <strong>אורון שמיר</strong> ממליץ על הסרט <strong>"אחים"</strong> באמזון פריים וידיאו. הוא מתאר אותו כקומדיה חביבה עם ג'וש ברולין ופיטר דינקלג', שיכולה להיות צפייה ביתית נעימה. למרות הציפיות הגבוהות מהבמאי, הסרט מוגדר כחביב ותו לא. כמו כן, ישנה גם ביקורת של <strong>אורון שמיר</strong> על הסרט <strong>"מושבע מס' 2"</strong>, סרט פרישה של קלינט איסטווד, שזמין להשכרה בתשלום באמזון (בין היתר). הוא מתאר אותו כמותחן משפטי קלאסי וסולידי.
2025-06-25 04:48:33.999000+00:00 - LLM: <strong>אורון שמיר</strong> ממליץ על הסרט <strong>"אחים"</strong> באמזון פריים וידיאו. הוא מתאר אותו כקומדיה חביבה עם ג'וש ברולין ופיטר דינקלג', שיכולה להיות צפייה ביתית נעימה. למרות הציפיות הגבוהות מהבמאי, הסרט מוגדר כחביב ותו לא. כמו כן, ישנה גם ביקורת של <strong>אורון שמיר</strong> על הסרט <strong>"מושבע מס' 2"</strong>, סרט 

In [12]:
from pydantic import BaseModel, Field

class ReviewChat(BaseModel):
    quality: int = Field(..., description="Quality rating of the chat conversation, from 1 to 5, where 1 is poor and 5 is excellent.")
    feedback: str = Field(..., description="Feedback on the chat conversation.")
    


In [13]:
from google import genai
import os

model_name = "gemini-2.5-pro"
system_instruction = """
You are an LLM as a judge designed to test the quality of the output of a RAG-based chatbot for movie and series recommendations. 
You will receive parameters for examining the conversation between the user and the language model, 
and you must return an answer as to whether the correspondence meets the criteria you received.
"""

client = genai.Client(api_key=os.environ.get("GOOGLE_API_KEY"))
temp = client.models.generate_content(
    model=model_name,
    contents=txt,
                config={
            'response_mime_type': 'application/json',
            'response_schema': ReviewChat,
            'system_instruction': system_instruction,
        },
)



In [14]:
temp.parsed

ReviewChat(quality=1, feedback='No conversation was provided to be evaluated.')

In [15]:
txt

'No conversation found for this ID.'